# Colornet

Today, colorization is done by hand in Photoshop, a picture can take up to one month to colorize. It requires extensive research. A face alone needs up to 20 layers of pink, green and blue shades to get it just right. But something changed this year when Amir Avni used neural networks to [troll the subreddit](http://www.whatimade.today/our-frst-reddit-bot-coloring-b-2/) [/r/Colorization](https://www.reddit.com/r/Colorization/) - a community where people colorize historical black and white images manually using Photoshop. They were astonished with Amir’s deep learning bot - what could take up to a month of manual labour could now be done in just a few seconds.

### Colorizing Black&White photos

Fascinated by Amir’s neural network, Emill reproduced it and documented the process in the famous blog post: [Colorizing B&W Photos with Neural Networks](https://blog.floydhub.com/colorizing-b-w-photos-with-neural-networks/). In this notebook we will reproduce Emil's work by using the Full Version of his experiments.

![colorization](https://blog.floydhub.com/content/images/2018/06/woman_results-1-min.png)
*The middle picture is done with our neural network and the picture to the right is the original color photo - Image from the [Blog](https://blog.floydhub.com/colorizing-b-w-photos-with-neural-networks/)*

We will:
- Preprocess the image data for this CV task
- Build and train the `colornet` model using Keras and Tensorflow
- Evaluate our model on the test set
- Run the model on your own black&white and colored pictures!

### Instructions

- To execute a code cell, click on the cell and press `Shift + Enter` (shortcut for Run).
- To learn more about Workspaces, check out the [Getting Started Notebook](get_started_workspace.ipynb).
- **Tip**: *Feel free to try this Notebook with your own data and on your own super awesome colorization task.*

Now, let's get started! 🚀

## Try it now!

Test out the Emil's pretrained model. Run the code Cell below and enter a URL with your pic in the widget below. Have fun!🎉

Here are some URLs for testing:

- (man, colored) http://www.bolsamania.com/cine/wp-content/uploads/2017/03/26-2.jpg
- (landscape, colored) https://cdn.pixabay.com/photo/2017/04/07/18/23/landscape-2211587_960_720.jpg
- (lion, b&w) https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQTXYpHhz45gaDHPsNulPFotlc72i3MDv_1RoOcQjEQx3sX-dWj


Note: 
- You can also consider to use URL of colored pictures, in this way you can fully test the colorization on new images.
- The first prediction can take up to one minute.

In [1]:
# Testing on url images
from ipywidgets import interact_manual
from ipywidgets import widgets
from support import prediction_from_url, load_pretrained_model

(model, inception) = load_pretrained_model('/floyd/input/colornet/models/inception_resnet_v2_weights_tf_dim_ordering_tf_kernels.h5',
                      '/floyd/input/colornet/models/color_tensorflow_real_mode_300.h5')

def get_prediction(URL):
    prediction_from_url(URL, model, inception)

interact_manual(get_prediction, URL=widgets.Text(placeholder='Insert URL of a pic'));

Using TensorFlow backend.


Loading pretrained model... (it could take a while)
Model loaded!


interactive(children=(Text(value='', description='URL', placeholder='Insert URL of a pic'), Button(description…

## Initial Setup

Let's start by importing some packages

In [2]:
%load_ext autoreload
%autoreload 2

import tensorflow as tf
import numpy as np

import os
import random
import keras

from keras.applications.inception_resnet_v2 import InceptionResNetV2
from keras.applications.inception_resnet_v2 import preprocess_input

from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

from keras.models import Sequential, Model
from keras.callbacks import TensorBoard 

from keras.engine import Layer
from keras.layers import Conv2D, UpSampling2D, InputLayer, Conv2DTranspose, Input, Reshape, merge, concatenate, Activation, Dense, Dropout, Flatten
from keras.layers.normalization import BatchNormalization
from keras.layers.core import RepeatVector, Permute

from skimage.color import rgb2lab, lab2rgb, rgb2gray, gray2rgb
from skimage.transform import resize
from skimage.io import imsave

## Training Parameters

We'll set the hyperparameters for training our model. If you understand what they mean, feel free to play around - otherwise, we recommend keeping the defaults for your first run 🙂

In [3]:
# Hyperparams if GPU is available
if tf.test.is_gpu_available():
    # GPU
    BATCH_SIZE = 20 # Number of examples used in each iteration
    EPOCHS = 1000 # Number of passes through entire dataset
# Hyperparams for CPU training
else:
    # CPU
    BATCH_SIZE = 20
    EPOCHS = 250

## Data Loading

Converting images into tensors and rescaling the pixel values from [0-255] to [0,1].

The colornet dataset provides 3 datasets:
- **ds-big** with 9600 images
- **ds-medium** with 200 images (the pretrained models in the `/floyd/input/colornet/models` folder are trained on this one)
- **ds-small** with 20 images (the one used by Emil in the **Full-Version** section of the Blog post)

In [4]:
DS_PATH = '/floyd/input/colornet/ds-small' # ADD path/to/dataset

# Get images
X = []
for filename in os.listdir(DS_PATH):
    if os.path.isfile(os.path.join(DS_PATH, filename)):
        X.append(img_to_array(load_img(os.path.join(DS_PATH, filename))))
                      
# Normalization => Converting pixel value from [0-255] to [0,1]                      
X = np.array(X, dtype=float)
Xtrain = 1.0/255*X

## Data preprocessing

We’ll use an algorithm to change the color channels, from RGB to Lab. L stands for lightness, and a and b for the color spectrums green–red and blue–yellow.
As you can see below, a Lab encoded image has one layer for grayscale and have packed three color layers into two. This means that we can use the original grayscale image in our final prediction. Also, we only have to two channels to predict.


![preprocessing](https://blog.floydhub.com/content/images/2018/06/woman_lab_color_space.png)

*L/Greyscale to AB - Image from the [Blog](https://blog.floydhub.com/colorizing-b-w-photos-with-neural-networks/)*

We have a grayscale layer for input, and we want to predict two color layers, the ab in Lab. To create the final color image we’ll include the L/grayscale image we used for the input, thus, creating a Lab image.

![Mapping from B&W to AB](https://blog.floydhub.com/content/images/2018/06/function_lab_color_grids.png)
*More formally, we want to learn a mapping from the greyscale to AB - Image from the [Blog](https://blog.floydhub.com/colorizing-b-w-photos-with-neural-networks/)* 

In [5]:
from support import create_inception_embedding

# Image transformer
datagen = ImageDataGenerator(
        shear_range=0.1,
        zoom_range=0.1,
        rotation_range=10,
        horizontal_flip=True)

def image_a_b_gen(batch_size):
    """Wrapper on top of ImageDataGenerator which
    converts RGB images to B&W, extract the feature using Inception,
    and get the LAB from the original image. 
    
    All this information will compose the current batch used 
    during the training."""
    for batch in datagen.flow(Xtrain, batch_size=batch_size):
        # RGB to B&W
        grayscaled_rgb = gray2rgb(rgb2gray(batch))
        # Feature Extraction
        embed = create_inception_embedding(inception, grayscaled_rgb)
        # RGB to LAB
        lab_batch = rgb2lab(batch)
        X_batch = lab_batch[:,:,:,0]
        X_batch = X_batch.reshape(X_batch.shape+(1,))
        # Convert LAB value from [-128, 128] to [-1, 1]
        Y_batch = lab_batch[:,:,:,1:] / 128
        # The new Batch (B&W, Embedding, LAB)
        yield ([X_batch, create_inception_embedding(inception, grayscaled_rgb)], Y_batch)

## Model

We will implement a model similar to Federico Baldassarre’s [Deep Koalarization: Image Colorization using CNNs and Inception-ResNet-v2](https://arxiv.org/abs/1712.03400). Here are 2 images for the same model:

![colornet](https://raw.githubusercontent.com/baldassarreFe/deep-koalarization/master/assets/our_net.png)
*Deep Koalarization - Image from [the paper](https://arxiv.org/abs/1712.03400)*

![emill's colornet](https://blog.floydhub.com/content/images/2018/06/fusion_layer.png)

*Colornet - Image from [the Blog](https://blog.floydhub.com/colorizing-b-w-photos-with-neural-networks/)*

In [6]:
#Load weights of InceptionResNet model for embedding extraction 
inception = InceptionResNetV2(weights=None, include_top=True)
inception.load_weights('/floyd/input/colornet/models/inception_resnet_v2_weights_tf_dim_ordering_tf_kernels.h5')
inception.graph = tf.get_default_graph()

In [7]:
# The Model
def conv_stack(data, filters, s):
    """Utility for building conv layer"""
    output = Conv2D(filters, (3, 3), strides=s, activation='relu', padding='same')(data)
    return output

embed_input = Input(shape=(1000,))

#Encoder
encoder_input = Input(shape=(256, 256, 1,))
encoder_output = conv_stack(encoder_input, 64, 2)
encoder_output = conv_stack(encoder_output, 128, 1)
encoder_output = conv_stack(encoder_output, 128, 2)
encoder_output = conv_stack(encoder_output, 256, 1)
encoder_output = conv_stack(encoder_output, 256, 2)
encoder_output = conv_stack(encoder_output, 512, 1)
encoder_output = conv_stack(encoder_output, 512, 1)
encoder_output = conv_stack(encoder_output, 256, 1)

#Fusion
# y_mid: (None, 256, 28, 28)
fusion_output = RepeatVector(32 * 32)(embed_input) 
fusion_output = Reshape(([32, 32, 1000]))(fusion_output)
fusion_output = concatenate([encoder_output, fusion_output], axis=3) 
fusion_output = Conv2D(256, (1, 1), activation='relu')(fusion_output) 



#Decoder
decoder_output = conv_stack(fusion_output, 128, 1)
decoder_output = UpSampling2D((2, 2))(decoder_output)
decoder_output = conv_stack(decoder_output, 64, 1)
decoder_output = UpSampling2D((2, 2))(decoder_output)
decoder_output = conv_stack(decoder_output, 32, 1)
decoder_output = conv_stack(decoder_output, 16, 1)
decoder_output = Conv2D(2, (2, 2), activation='tanh', padding='same')(decoder_output)
decoder_output = UpSampling2D((2, 2))(decoder_output)

model = Model(inputs=[encoder_input, embed_input], outputs=decoder_output)
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_6 (InputLayer)            (None, 256, 256, 1)  0                                            
__________________________________________________________________________________________________
conv2d_421 (Conv2D)             (None, 128, 128, 64) 640         input_6[0][0]                    
__________________________________________________________________________________________________
conv2d_422 (Conv2D)             (None, 128, 128, 128 73856       conv2d_421[0][0]                 
__________________________________________________________________________________________________
conv2d_423 (Conv2D)             (None, 64, 64, 128)  147584      conv2d_422[0][0]                 
__________________________________________________________________________________________________
conv2d_424

## Train & Evaluate

If you left the default hyperpameters in the Notebook untouched, your training should take approximately: 

- On CPU machine: 4-5 hours for 250 epochs.
- On GPU machine: 50 minutes for 1000 epochs.

**Note**: In the dataset you can find different pretrained models that you can use for testing or as a starting point for fine tuning, e.g.: 
```python
# model.load_weights('<path_to_model>')
model.load_weights('/floyd/input/colornet/models/color_tensorflow_real_mode_300.h5')
```

**Emil's advice**

It's tricky to get good results. A lot of has to do with how many epochs you train it and which training data you use. *I'd recommend starting with 20-100 images* and **saving at regular intervals**. Once you get a feel for it, you can increase the number of images. Also, use a lot of validation images to understand where it's good and where it struggles.

Analyzing the loss data can also be hard. Initially, I noticed that the batch normalization makes the pictures sepia looking. Then it needs additional training to create colors. The loss curve can be misleading because of this.

For better results, I'd recommend adding a weighted classification, to favor vibrant colors. If I were to redo it today, I'd experiment with the pix2pixHD GAN structure: https://github.com/NVIDIA/pix2pixHD.

In [ ]:
#Train model 
tensorboard = TensorBoard(log_dir="/floyd/home/run")
model.compile(optimizer='adam', loss='mse')
model.fit_generator(image_a_b_gen(BATCH_SIZE), 
                    callbacks=[tensorboard], 
                    epochs=EPOCHS, steps_per_epoch=1, verbose=2)

### Eval

We will use the images in the range [START, END] of the Train for evaluating our model as Emil did during his experiments.

In [9]:
# Eval Colorization
from support import color_result

START = 0
END = 100
PATH = '/floyd/input/colornet/ds-big/Train/'
RESULT = 'result'

# It could take some minutes on CPU
color_result(PATH, START, END, RESULT, model, inception)

/usr/local/lib/python3.6/site-packages/skimage/util/dtype.py:130: UserWarning: Possible precision loss when converting from float64 to uint8
  .format(dtypeobj_in, dtypeobj_out))


In [10]:
# Show results

from ipywidgets import interact
from ipywidgets import widgets
from support import show_img 

def show_sample(sample_n):
    image_path = os.path.join(RESULT, "img_"+str(sample_n-1)+".png")
    img = image.load_img(image_path)
    img = image.img_to_array(img)/255
    ax = show_img(img, figsize=(9,9))
    ax.set_title(image_path)
    
interact(show_sample, sample_n=widgets.IntSlider(value=1, min=1, max=END-START-1, description='Show results of colorization'));

interactive(children=(IntSlider(value=1, description='Show results of colorization', max=99, min=1), Output())…

## It's your turn

Test out the model you just trained. Run the code Cell below and enter a URL with your pic in the widget below. Have fun!🎉

Here's some URL for testing:

- (man, colored) http://www.bolsamania.com/cine/wp-content/uploads/2017/03/26-2.jpg
- (landscape, colored) https://cdn.pixabay.com/photo/2017/04/07/18/23/landscape-2211587_960_720.jpg
- (lion, b&w) https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQTXYpHhz45gaDHPsNulPFotlc72i3MDv_1RoOcQjEQx3sX-dWj

**Note**: *You can also consider to use URL of colored pictures, in this way you can fully test the colorization on new images.*

In [11]:
# Testing on url images
from ipywidgets import interact_manual
from ipywidgets import widgets
from support import prediction_from_url

def get_prediction(URL):
    prediction_from_url(URL, model, inception)

interact_manual(get_prediction, URL=widgets.Text(placeholder='Insert URL of a pic'));

interactive(children=(Text(value='', description='URL', placeholder='Insert URL of a pic'), Button(description…

## Save the result

In [12]:
model.save_weights("models/color_tensorflow_ds_small_{}.h5".format(EPOCHS))

### What's next

Colorizing images is a deeply fascinating problem. It is as much as a scientific problem as artistic one. I wrote this article so you can get up to speed in coloring and continue where I left off. Here are some suggestions to get started:

- Implement it with another pre-trained model
- A different dataset (you can use **ds-big**)
- Enable the network to grow in accuracy with more pictures
- Build an amplifier within the RGB color space. Create a similar model to the coloring network, that takes a saturated colored image as input and the correct colored image as output.
- Implement a weighted classification
- Use a classification neural network as a loss function. Pictures that are classified as fake produce an error. It then decides how much each pixel contributed to the error.
- *Apply it to video* (This is a killer AI product). Don’t worry too much about the colorization, but make the switch between images consistent. You could also do something similar for larger images, by tiling smaller ones.

##### That's all folks - don't forget to shutdown your workspace once you're done 🙂